# Features Selection 1

In [343]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder

from src.helpers import *

In [344]:
df = pd.read_csv('../data/train.csv')

In [345]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

# ---Data Cleaning---

In [346]:
# Converting integers to string for the OrdinalEncoder

def convert_int_to_string(data, columns):
  for column in columns:
    data[column] = data[column].astype(str)
  return data

In [347]:
columns_to_convert = ['OverallQual', 'OverallCond']

df = convert_int_to_string(df, columns_to_convert)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   object 
 18  OverallC

In [348]:
# OutlierRemover is a custom transformer imported from src/helpers.py
# It can remove outliers from multiple columns by specifying the iqr_multipler

outlier_cols = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea',
                'OpenPorchSF']

outlier_remover = OutlierRemover(columns=outlier_cols,iqr_multiplier=100)

outlier_remover.fit(df)
df_proc = outlier_remover.transform(df)

df_proc.count()

Id               1460
MSSubClass       1460
MSZoning         1460
LotFrontage      1201
LotArea          1460
Street           1460
Alley              91
LotShape         1460
LandContour      1460
Utilities        1460
LotConfig        1460
LandSlope        1460
Neighborhood     1460
Condition1       1460
Condition2       1460
BldgType         1460
HouseStyle       1460
OverallQual      1460
OverallCond      1460
YearBuilt        1460
YearRemodAdd     1460
RoofStyle        1460
RoofMatl         1460
Exterior1st      1460
Exterior2nd      1460
MasVnrType        588
MasVnrArea       1452
ExterQual        1460
ExterCond        1460
Foundation       1460
BsmtQual         1423
BsmtCond         1423
BsmtExposure     1422
BsmtFinType1     1423
BsmtFinSF1       1460
BsmtFinType2     1422
BsmtFinSF2       1460
BsmtUnfSF        1460
TotalBsmtSF      1460
Heating          1460
HeatingQC        1460
CentralAir       1460
Electrical       1459
1stFlrSF         1460
2ndFlrSF         1460
LowQualFin

# ---Pipeline---

In [349]:
# # Defining the list of columns that will go into the pipeline
# # Remove columns that should not be included in the pipeline

# all_columns = df.columns.tolist()
# columns_remove_from_filter = ['SalePrice','Id','Street','Utilities','LowQualFinSF','GarageYrBlt',
#                               '3SsnPorch']

# columns_to_keep = [item for item in all_columns if item not in columns_remove_from_filter]

In [350]:
columns_to_keep = ['1stFlrSF','BsmtFinSF1','BsmtQual','ExterQual','GarageArea','GarageCars','GrLivArea','KitchenQual','LotArea','OverallQual',
                   'TotalBsmtSF','YearBuilt','YearRemodAdd','Fireplaces','FireplaceQu','OpenPorchSF']

In [351]:
# Preprocessor to filter unwanted columns, or unexpected columns.

preprocessor_filter = ColumnTransformer([
  ('passthrough','passthrough',columns_to_keep)
],remainder='drop', verbose_feature_names_out=False)

In [352]:
# Preprocessor for imputing all null variables

# cols_impute_na = ['Alley', 'GarageType', 'Fence','BsmtQual', 'BsmtCond', 'BsmtExposure',
#                   'BsmtFinType1','BsmtFinType2', 'FireplaceQu', 'GarageFinish', 'GarageQual','GarageCond',
#                   'PoolQC', 'MiscFeature']
# cols_impute_none = ['MasVnrType']
# cols_impute_most_freq = ['Electrical']
# cols_impute_zero = ['LotFrontage', 'MasVnrArea']

cols_impute_na = ['BsmtQual']
# cols_impute_none = ['MasVnrType']
# cols_impute_most_freq = ['Electrical']
# cols_impute_zero = ['LotFrontage', 'MasVnrArea']

preprocessor_imputer = ColumnTransformer([
  ('impute_na', SimpleImputer(strategy='constant', fill_value='NA'), cols_impute_na),
  # ('impute_none', SimpleImputer(strategy='constant', fill_value='None'), cols_impute_none),
  # ('impute_zero', SimpleImputer(strategy='constant', fill_value=0), cols_impute_zero),
  # ('impute_most_freq', SimpleImputer(strategy='most_frequent'), cols_impute_most_freq)
], remainder='passthrough', verbose_feature_names_out=False)

In [353]:
# Defining maps for Ordinal categories

dict_na_ex_6 = {'NA':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
dict_na_gd_5 = {'NA':0,'No':1,'Mn':2,'Av':3,'Gd':4}
dict_bsmt = {'NA':0,'Unf':1,'LwQ':2,'Rec':3,'BLQ':4,'ALQ':5,'GLQ':6}
dict_garage = {'NA':0,'Unf':1,'RFn':2,'Fin':3}
dict_1_10 = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5, 7:6, 8:7, 9:8, 10:9}
dict_fence = {'NA':0, 'MnWw':1, 'GdWo':2, 'MnPrv':3, 'GdPrv':4}


ordinal_cat_map = [
  {'col':'BsmtQual','mapping':dict_na_ex_6},
  {'col':'BsmtCond','mapping':dict_na_ex_6},
  {'col':'BsmtExposure','mapping':dict_na_gd_5},
  {'col':'BsmtFinType1','mapping':dict_bsmt},
  {'col':'BsmtFinType2','mapping':dict_bsmt},
  {'col':'FireplaceQu','mapping':dict_na_ex_6},
  {'col':'GarageFinish','mapping':dict_garage},
  {'col':'GarageQual','mapping':dict_na_ex_6},
  {'col':'GarageCond','mapping':dict_na_ex_6},
  {'col':'PoolQC','mapping':dict_na_ex_6},
  {'col': 'OverallQual', 'mapping':dict_1_10},
  {'col': 'OverallCond', 'mapping':dict_1_10},
  {'col': 'ExterQual', 'mapping': dict_na_ex_6},
  {'col': 'ExterCond', 'mapping': dict_na_ex_6},
  {'col': 'HeatingQC', 'mapping': dict_na_ex_6},
  {'col': 'KitchenQual', 'mapping': dict_na_ex_6},
  {'col': 'Fence', 'mapping': dict_fence}
  ]

In [354]:
# Preprocessor for encoding and scaling

# ohe_cols = ['MSSubClass','MSZoning','Alley','LotShape','LandContour','LotConfig','LandSlope',
#             'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st',
#             'Exterior2nd','MasVnrType','Foundation','Heating', 'CentralAir','Electrical','Functional',
#             'GarageType','PavedDrive','MiscFeature','MoSold','YrSold','SaleType','SaleCondition']

# oe_cols = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
#            'BsmtFinType1','BsmtFinType2','HeatingQC','FireplaceQu', 'GarageFinish','GarageQual', 'GarageCond',
#            'PoolQC','KitchenQual','Fence']

# robust_cols = ['LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF',
#                'GrLivArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','ScreenPorch','PoolArea','MiscVal']


oe_cols = ['OverallQual','ExterQual','BsmtQual','KitchenQual','FireplaceQu']

robust_cols = ['LotArea','BsmtFinSF1','TotalBsmtSF','1stFlrSF','GrLivArea','OpenPorchSF']


preprocessor_encode_scale = ColumnTransformer([
  # ('ohe', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), ohe_cols),
  ('oe', OrdinalEncoder(), oe_cols),
  ('standard_scaler', StandardScaler(), ['GarageArea']),
  ('robust_scaler', RobustScaler(), robust_cols),
  ('minmax_scaler', MinMaxScaler(), ['YearBuilt','YearRemodAdd'])
], remainder='passthrough', verbose_feature_names_out=False)

In [355]:
X = df_proc.drop(columns='SalePrice')
y = df_proc['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [356]:
main_pipe = Pipeline([
  ('preprocessor_filter', preprocessor_filter),
  ('preprocessor_imputer', preprocessor_imputer),
  ('preprocessor_encode_scale', preprocessor_encode_scale)
])

In [357]:
main_pipe.fit(X_train)

X_train_proc = main_pipe.transform(X_train)
X_val_proc = main_pipe.transform(X_val)

In [358]:
len(X_train_proc.columns)

16

### Random Forest Regressor

In [359]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train_proc, y_train)

linear_results_train = evaluate_regression(rf, X_train_proc, y_train, 'rf')
linear_results_val = evaluate_regression(rf, X_val_proc, y_val, 'rf')

In [360]:
linear_results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf,6741.582583,1.280199e+08,11314.589082,0.040898,0.978537,0.978238


In [361]:
linear_results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
rf,17008.649463,7.861016e+08,28037.503417,0.109088,0.897514,0.891551


In [362]:
val_preds = rf.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.16139440657695994

In [363]:
# Get feature importances
importances = rf.feature_importances_

# Get feature names (from your dataset)
feature_names = X_train_proc.columns

# Create a DataFrame to pair feature names with their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

In [364]:
pd.set_option('display.max_rows', 300)

In [365]:
feature_importance_df.sort_values(by='Feature')

,Feature,Importance
9,1stFlrSF,0.039950
7,BsmtFinSF1,0.059403
2,BsmtQual,0.029192
1,ExterQual,0.011832
4,FireplaceQu,0.011957
15,Fireplaces,0.003954
5,GarageArea,0.028566
14,GarageCars,0.179445
10,GrLivArea,0.198107
3,KitchenQual,0.017102
